In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial

import glam
import parameter_recovery as pr

%load_ext autoreload
%autoreload 2

# Systematic parameter recovery – Hierarchical model, with ADVI

In [ ]:
output_folder = join('results', 'parameter-recovery_hierarchical', 'advi')
label = 'hierarchical_advi'

## Define parameter ranges from 2011 estimates

In [ ]:
bounds = dict(v=[0.000015, 0.00015],
              gamma=[-1, 1],
              s=[0.004, 0.011],
              tau=[0.1, 1.25])

In [ ]:
# Determine low, medium, high mean values as 10th, 50th and 90th percentile of these ranges
means = {parameter: np.percentile(np.linspace(*bounds[parameter], 100), [10, 50, 90])
          for parameter in ['v', 'gamma', 's', 'tau']}
means

In [ ]:
sds = dict(v=0.00005,
           s=0.001,
           gamma=0.3,
           tau=0.2)

In [ ]:
parameter_info = {parameter: dict(mu=dict(low=means[parameter][0],
                                          medium=means[parameter][1],
                                          high=means[parameter][2]),
                                  sd=sds[parameter],
                                  bounds=tuple(bounds[parameter]))
                  for parameter in ['v', 'gamma', 's', 'tau']}

## Run the recovery in parallel

In [ ]:
partial_output_folder = join(output_folder, 'partial')

simulate_group_args = dict(kind='hierarchical',
                           n_individuals=20,
                           n_trials=50, 
                           n_items=3,
                           error_weight=0,
                           value_range=[1, 10],
                           seed=None)

make_model_args = dict(kind='hierarchical',
                       t0_val=0,
                       error_weight=0)

fit_args = dict(method='VI',
                draws=2000,
                n_vi=50000,
                progressbar=False)

wrapper = partial(pr.recover_and_save_hierarchical,
                  output_folder=partial_output_folder,
                  label=label,
                  simulate_group_args=simulate_group_args,
                  make_model_args=make_model_args,
                  fit_args=fit_args)

In [ ]:
from multiprocessing import Pool

n_cores = 8
p = Pool(n_cores)

output = p.map(wrapper, pr.generate_hierarchical_parameter_sets(parameter_info))

INFO (theano.gof.compilelock): Waiting for existing lock by process '27036' (I am process '27032')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27036' (I am process '27035')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27036' (I am process '27030')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27036' (I am process '27034')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x